In [2]:
import numpy as np
import torch

In [6]:
train_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/embeddings_train.pt'
train_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/labels_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/embeddings_val.pt'
val_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/labels_val.pt'


train_embeddings = torch.load(train_embeddings_path)
train_labels = torch.load(train_labels_path)

val_embeddings = torch.load(val_embeddings_path)
val_labels = torch.load(val_labels_path)

train_embeddings_np = train_embeddings.numpy()
train_labels_np = train_labels.numpy()

val_embeddings_np = val_embeddings.numpy()
val_labels_np= val_labels.numpy()




In [10]:
print(train_embeddings_np.shape)
print(train_labels_np.shape)

print(val_embeddings_np.shape)
print(val_labels_np.shape)

(1043, 4560)
(1043, 1)
(252, 4560)
(252, 1)


In [21]:
import torch
import numpy as np

train_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/embeddings_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/embeddings_val.pt'

def load_embeddings(path):
    return torch.load(path).numpy()

A = load_embeddings(train_embeddings_path)
B = load_embeddings(val_embeddings_path)

def chamfer_distance(A, B):
    A_to_B = np.mean([np.min(np.sum((B - a) ** 2, axis=1)) for a in A])
    
    B_to_A = np.mean([np.min(np.sum((A - b) ** 2, axis=1)) for b in B])
    
    return A_to_B + B_to_A

distance = chamfer_distance(A, B)
print(f'Chamfer Distance: {distance}')


Chamfer Distance: 29.32785415649414


In [1]:
train_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/embeddings_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/embeddings_val.pt'
train_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/labels_train.pt'
val_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/labels_val.pt'

embeddings = torch.load(embeddings_path).numpy()
labels = torch.load(labels_path).numpy()


NameError: name 'torch' is not defined

# Chamfer distance con los vectores

In [25]:
import torch
import numpy as np

# Cargar los embeddings y las etiquetas desde los archivos .pt
train_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/embeddings_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/embeddings_val.pt'
train_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/train/labels_train.pt'
val_labels_path = '/data/ProstateContrastiveSPD/embeddings/fold_5/val/labels_val.pt'

# Función para cargar los datos de los archivos .pt
def load_data(embeddings_path, labels_path):
    embeddings = torch.load(embeddings_path).numpy()
    labels = torch.load(labels_path).numpy()
    return embeddings, labels

# Función para calcular la Chamfer Distance según la fórmula proporcionada
def chamfer_distance(A, B):
    A_to_B = np.mean([np.min(np.sum((B - a) ** 2, axis=1)) for a in A])
    B_to_A = np.mean([np.min(np.sum((A - b) ** 2, axis=1)) for b in B])
    return A_to_B + B_to_A

# Cargar los embeddings y las etiquetas
train_embeddings, train_labels = load_data(train_embeddings_path, train_labels_path)
val_embeddings, val_labels = load_data(val_embeddings_path, val_labels_path)

# Aplanar las etiquetas para eliminar la dimensión extra
train_labels = train_labels.flatten()
val_labels = val_labels.flatten()

# Filtrar los embeddings por clase
train_class_0 = train_embeddings[train_labels == 0]
train_class_1 = train_embeddings[train_labels == 1]
val_class_0 = val_embeddings[val_labels == 0]
val_class_1 = val_embeddings[val_labels == 1]

# Calcular Chamfer Distance para los diferentes conjuntos
distances = {
    "train_class_0_vs_train_class_1": chamfer_distance(train_class_0, train_class_1),
    "train_class_0_vs_val_class_1": chamfer_distance(train_class_0, val_class_1),
    "train_class_0_vs_val_class_0": chamfer_distance(train_class_0, val_class_0),
    "val_class_0_vs_val_class_1": chamfer_distance(val_class_0, val_class_1),
    "train_class_1_vs_val_class_1": chamfer_distance(train_class_1, val_class_1)
    
}

# Imprimir las distancias calculadas
for key, value in distances.items():
    print(f'Chamfer Distance ({key}): {value}')


Chamfer Distance (train_class_0_vs_train_class_1): 35.016605377197266
Chamfer Distance (train_class_0_vs_val_class_1): 38.831703186035156
Chamfer Distance (train_class_0_vs_val_class_0): 29.159626007080078
Chamfer Distance (val_class_0_vs_val_class_1): 41.90660858154297
Chamfer Distance (train_class_1_vs_val_class_1): 33.93451690673828


33.93451690673828

# Riemannian distance

In [2]:
import os
import numpy as np
from scipy.linalg import sqrtm, logm, inv, norm
import torch

# Configuración de la GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["OMP_NUM_THREADS"] = str(1)

# Rutas de los archivos
train_embeddings_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/train/embeddingsSPD_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/val/embeddingsSPD_val.pt'
train_labels_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/train/labelsSPD_train.pt'
val_labels_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/val/labelsSPD_val.pt'

# Verificar el dispositivo
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Usando el dispositivo: {device}')

# Función para cargar los datos de los archivos .pt
def load_data(embeddings_path, labels_path):
    embeddings = torch.load(embeddings_path).numpy()
    labels = torch.load(labels_path).numpy()
    return embeddings, labels

# Función para calcular la distancia de Riemann entre dos matrices SPD
def riemannian_distance(A, B):
    sqrt_A = sqrtm(A)
    inv_sqrt_A = np.linalg.inv(sqrt_A)
    C = inv_sqrt_A @ B @ inv_sqrt_A
    log_C = logm(C)
    return norm(log_C, 'fro')

# Función para calcular la Chamfer Distance usando distancia de Riemann
def chamfer_distance_riemannian(S1, S2):
    S1_to_S2 = np.mean([np.min([riemannian_distance(x1, x2) for x2 in S2]) for x1 in S1])
    S2_to_S1 = np.mean([np.min([riemannian_distance(x2, x1) for x1 in S1]) for x2 in S2])
    return S1_to_S2 + S2_to_S1

# Cargar los embeddings y las etiquetas
train_embeddings, train_labels = load_data(train_embeddings_path, train_labels_path)
val_embeddings, val_labels = load_data(val_embeddings_path, val_labels_path)

# Aplanar las etiquetas para eliminar la dimensión extra
train_labels = train_labels.flatten()
val_labels = val_labels.flatten()

# Filtrar los embeddings por clase
train_class_0 = train_embeddings[train_labels == 0]
train_class_1 = train_embeddings[train_labels == 1]
val_class_0 = val_embeddings[val_labels == 0]
val_class_1 = val_embeddings[val_labels == 1]

# Calcular la Chamfer Distance usando distancia de Riemann para los diferentes conjuntos
distances = {
    "train_class_0_vs_train_class_1": chamfer_distance_riemannian(train_class_0, train_class_1),
    "train_class_0_vs_val_class_1": chamfer_distance_riemannian(train_class_0, val_class_1),
    "train_class_0_vs_val_class_0": chamfer_distance_riemannian(train_class_0, val_class_0),
    "val_class_0_vs_val_class_1": chamfer_distance_riemannian(val_class_0, val_class_1),
    "train_class_1_vs_val_class_1": chamfer_distance_riemannian(train_class_1, val_class_1)
}

# Imprimir las distancias calculadas
for key, value in distances.items():
    print(f'Riemannian Chamfer Distance ({key}): {value}')


Usando el dispositivo: cuda:0


KeyboardInterrupt: 

# Log-Euclidean metric

In [7]:
import os
import numpy as np
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["OMP_NUM_THREADS"] = str(1)

train_embeddings_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/train/embeddingsSPD_train.pt'
val_embeddings_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/val/embeddingsSPD_val.pt'
train_labels_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/train/labelsSPD_train.pt'
val_labels_path = '/data/ProstateContrastiveSPD/embeddingsSPD/fold_5/val/labelsSPD_val.pt'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Usando el dispositivo: {device}')

# Función para cargar los datos de los archivos .pt
def load_data(embeddings_path, labels_path):
    embeddings = torch.load(embeddings_path).numpy()
    labels = torch.load(labels_path).numpy()
    return embeddings, labels

# Función para calcular la distancia Log-Euclidean entre dos matrices SPD
def log_euclidean_distance(A, B):
    # Descomposición espectral
    eigvals_A, eigvecs_A = np.linalg.eigh(A)
    eigvals_B, eigvecs_B = np.linalg.eigh(B)
    
    # Calcular el logaritmo de los autovalores
    log_eigvals_A = np.diag(np.log(eigvals_A))
    log_eigvals_B = np.diag(np.log(eigvals_B))
    
    # Reconstruir las matrices logarítmicas
    log_A = eigvecs_A @ log_eigvals_A @ eigvecs_A.T
    log_B = eigvecs_B @ log_eigvals_B @ eigvecs_B.T
    
    # Calcular la norma de la diferencia
    return np.linalg.norm(log_A - log_B)

# Función para calcular la Chamfer Distance usando Log-Euclidean Metric
def chamfer_distance_log_euclidean(S1, S2):
    S1_to_S2 = np.mean([np.min([log_euclidean_distance(x1, x2) for x2 in S2]) for x1 in S1])
    S2_to_S1 = np.mean([np.min([log_euclidean_distance(x2, x1) for x1 in S1]) for x2 in S2])
    return S1_to_S2 + S2_to_S1

# Cargar los embeddings y las etiquetas
train_embeddings, train_labels = load_data(train_embeddings_path, train_labels_path)
val_embeddings, val_labels = load_data(val_embeddings_path, val_labels_path)

# Aplanar las etiquetas para eliminar la dimensión extra
train_labels = train_labels.flatten()
val_labels = val_labels.flatten()

# Filtrar los embeddings por clase
train_class_0 = train_embeddings[train_labels == 0]
train_class_1 = train_embeddings[train_labels == 1]
val_class_0 = val_embeddings[val_labels == 0]
val_class_1 = val_embeddings[val_labels == 1]

# Calcular la Chamfer Distance usando Log-Euclidean Metric para los diferentes conjuntos
distances = {
    "train_class_0_vs_train_class_1": chamfer_distance_log_euclidean(train_class_0, train_class_1),
    "train_class_0_vs_val_class_1": chamfer_distance_log_euclidean(train_class_0, val_class_1),
    "train_class_0_vs_val_class_0": chamfer_distance_log_euclidean(train_class_0, val_class_0),
    "val_class_0_vs_val_class_1": chamfer_distance_log_euclidean(val_class_0, val_class_1),
    "train_class_1_vs_val_class_1": chamfer_distance_log_euclidean(train_class_1, val_class_1)
}

# Imprimir las distancias calculadas
for key, value in distances.items():
    print(f'Log-Euclidean Chamfer Distance ({key}): {value}')


Usando el dispositivo: cuda:0
Log-Euclidean Chamfer Distance (train_class_0_vs_train_class_1): 12.148919334674682
Log-Euclidean Chamfer Distance (train_class_0_vs_val_class_1): 12.871937769008152
Log-Euclidean Chamfer Distance (train_class_0_vs_val_class_0): 11.087840384821785
Log-Euclidean Chamfer Distance (val_class_0_vs_val_class_1): 13.360884936807714
Log-Euclidean Chamfer Distance (train_class_1_vs_val_class_1): 11.975105139604086
